In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

Getting the source webpage


In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

Make cs_writer for the data scaped

In [3]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

Scaping the table in the webpage

In [4]:
table = soup.find('table', class_ = 'wikitable sortable')
rows = table.find_all('tr')

postcodes = [] #postcodes list
boroughs = [] #boroughs list
neighbourhoods = [] #neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  #to ignore cells with a borough that is Not assigned
            
            postcode = columns[0].text.split('\n')[0] #to remove in the end the newline character
            postcodes.append(postcode)
            
            borough = columns[1].text.split('\n')[0]
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] 
            
                      
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : #to skip the first row
        pass 
    
postcode_explored = [] #explored postcodes list
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] #Concat neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) #Writing the rows on the csv file
        postcode_explored.append(postcode_i)

Closing the CSV File

In [5]:
csv_file.close()

Laodingthe DATAFRAME

In [6]:
df = pd.read_csv('toronto_postal_codes.csv')

Clean the dataframe

In [7]:
#Drop "Not Assigned" Borough
df = df[df.Borough != 'Not assigned']

# Replace "Not Assigned" Neighbourhood with Borough value
df['Neighbourhood'] = df.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'], axis=1)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


DataFrame Shape

In [8]:
df.shape

(103, 3)